<h1 align="center">
  Python Benchmark Functions for Optimization
</h1>

[![Python](https://img.shields.io/pypi/pyversions/py_benchmark_functions.svg)](https://badge.fury.io/py/py_benchmark_functions)
[![PyPI](https://badge.fury.io/py/py_benchmark_functions.svg)](https://badge.fury.io/py/py_benchmark_functions)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/moesio-f/moesio-f/py-benchmark-functions/blob/main/examples/intro.ipynb)

[`py-benchamrk-functions`](https://github.com/moesio-f/py-benchmark-functions) is a simple library that provides benchmark functions for global optimization. It exposes implementations in major computing frameworks such as NumPy, TensorFlow and PyTorch. All implementations support `batch`-evaluation of coordinates, allowing for performatic evaluation of candidate solutions in the search space. The main goal of this library is to provide up-to-date implementations of multiple common benchmark functions in the scientific literature.

## Installation

Start by installing the library using your preferred package manager:

In [ ]:
try:
  import google.colab
  %pip install --upgrade pip uv
  !python -m uv pip install --reinstall py_benchmark_functions[tensorflow,torch]
  print("\033[31m [py_benchmark_functions] Restart runtime before proceeding.")
  exit(0)
except ImportError:
  pass

In [ ]:
import py_benchmark_functions as bf

print(bf.available_backends())
# Output: {'numpy', 'tensorflow', 'torch'}

print(bf.available_functions())
# Output: ['Ackley', ..., 'Zakharov']

## Instantiating and using Functions

The library is designed with the following entities:
- [`core.Function`](py_benchmBohachevskyark_functions/core/function.py): class that represents a benchmark function. An instance of this class represents an instance of the becnmark function for a given domain ([`core.Domain`](py_benchmBohachevskyark_functions/core/function.py)) and number of dimensions/coordinates.
- [`core.Transformation`](py_benchmBohachevskyark_functions/core/function.py): class that represents a _transformed_ (i.e., shifted, scaled, etc) function. It allows for programatically building new functions from existing ones.
- [`core.Metadata`](py_benchmBohachevskyark_functions/core/metadata.py): class thata represent _metadata_ about a given function (i.e., known global optima, default search space, default parameters, etc). A transformation inherits such metadata from the base function.

The benchmark functions can be instantiated in 3 ways:

1. Directly importing from `py_benchmark_functions.imp.{numpy,tensorflow,torch}` (e.g., `from py_benchmark_functions.imp.numpy import AckleyNumpy`);

In [ ]:
from py_benchmark_functions.imp.numpy import AckleyNumpy

fn = AckleyNumpy(dims=2)
print(fn.name, fn.domain)
print(fn.metadata)

2. Using the global `get_fn`, `get_np_function` or `get_tf_function` from `py_benchmark_functions`;

In [ ]:
import py_benchmark_functions as bf

fn = bf.get_fn("Zakharov", 2)
print(fn, type(fn))

fn1 = bf.get_np_function("Zakharov", 2)
print(fn1, type(fn1))

fn2 = bf.get_tf_function("Zakharov", 2)
print(fn2, type(fn2))

fn3 = bf.get_torch_function("Zakharov", 2)
print(fn3, type(fn3))

3. Using the [`Builder`](py_benchmBohachevskyark_functions/factory/builder.py) class;

In [ ]:
from py_benchmark_functions import Builder

fn = Builder().function("Alpine2").dims(4).transform(vshift=1.0).tensorflow().build()
print(fn, type(fn))

Regardless of how you get an instance of a function, all of them define the `__call__` method, which allows them to be called directly. Every `__call__` receives an `x` as argument (for NumPy, `x` should be an `np.ndarray`, for Tensorflow a `tf.Tensor`, and for PyTorch a `torch.Tensor`). The shape of `x` can either be `(batch_size, dims)` or `(dims,)`, while the output is `(batch_size,)` or `()` (a scalar). Those properties are illustrated below:

In [ ]:
import py_benchmark_functions as bf
import numpy as np

fn = bf.get_fn("Ackley", 2)
x = np.array([0.0, 0.0], dtype=np.float32)

print(fn(x))

x = np.expand_dims(x, axis=0)
print(x, fn(x))

x = np.repeat(x, 3, axis=0)
print(x, fn(x))

Additionally, for the `torch` and `tensorflow` backends, it is possible to use their `autograd` to differentiate any of the functions. Specifically, they expose the methods `.grads(x) -> Tensor` and `.grads_at(x) -> Tuple[Tensor, Tensor]` which returns the gradients for the input `x` and, for `grads_at`, the value of the function at `x` (in this order).

In [ ]:
import torch

fn = bf.get_torch_function("Ackley", 2)
x = torch.tensor([1.0, 2.0], dtype=torch.float32)

print(fn(x))
print(fn.grads(x))
print(fn.grads_at(x))

Beware that some functions are not continuously differentiable, which might return `NaN`'s values! For the specifics of how those backends handle such cases one should refer to the respective official documentation (see [A Gentle Introduction to `torch.autograd`](https://docs.pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html) and [Introduction to gradients and automatic differentiation](https://www.tensorflow.org/guide/autodiff)).

In [ ]:
x = torch.tensor([0.0, 0.0], dtype=torch.float32)

print(fn.grads(x))
print(fn.grads_at(x))

## Plotting and Drawing

Additionally, the [`Drawer`](py_benchmark_functions/plot/drawer.py) utility allows for plotting functions.

In [ ]:
from py_benchmark_functions.plot import Drawer

drawer = Drawer("Ackley")
drawer.show()

It is also possible to define custom domains.

In [ ]:
from py_benchmark_functions import Builder

fn = Builder().function("Ackley").dims(2).domain(domain_min=-1.0, domain_max=1.0).build()
drawer = Drawer(fn)
drawer.show()